In [ ]:
import pandas as pd
import numpy as np
import sklearn
import pickle
import seaborn as sns

from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer

from sklearn import model_selection
from sklearn import linear_model, svm, naive_bayes, neighbors, ensemble, metrics
from sklearn import metrics

from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier

from imblearn.over_sampling import SMOTE, ADASYN

from itertools import product
import util
from util.modeling import prepro, cross_val, get_scores, get_best_params, train_best_model, tune_hyper

import xgboost as xgb
from xgboost import XGBClassifier

In [ ]:
data = pd.read_csv("test_data.csv")

X = data.drop(labels=["grad_bach", "survey_weight"], axis=1)
y = data["grad_bach"]
weights = data["survey_weight"]

In [ ]:
# is there a way to select y_test so it's a good representation of the population?
# if we select poorly it might be a bad one even weighted properly
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.2, random_state=0, stratify=y)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.2, random_state=0, stratify=y_train_val)

In [ ]:
# https://scikit-learn.org/stable/modules/compose.html#pipeline

## Tune Hyperparameters for Logistic Regression

In [ ]:
C = [0.05, 0.1, 1, 5, 10, 100, 1000]
solver = ["liblinear"]
penalty = ['l1', 'l2']
class_weight = [None, "balanced"]

# fix oversampling w/ sample weights
samplers = [None]+ [SMOTE(random_state=0, k_neighbors=k) for k in [3, 5, 7]]
scale = [True]

keywords = ["C", "solver", "penalty", "class_weight"]
param_grid = product(C, solver, penalty, class_weight)

prod_size = len(C)*len(solver)*len(penalty)*len(class_weight)*len(samplers)*len(scale)
print_prog = (15, prod_size)

sc_lr = tune_hyper(X_train, y_train, LogisticRegression, keywords, param_grid, weights, samplers=samplers, scaling=scale, print_prog=print_prog)

In [ ]:
get_best_params(sc_lr, "fp5")

In [ ]:
keywords_lr = ["C", "solver", "penalty", "class_weight"]
lr_best = train_best_model(X_train, y_train, LogisticRegression, keywords_lr, sc_lr, "fp5", weights)
cols = prepro(X_train)[0].columns
lr_coefs = list(zip(cols, lr_best.coef_[0]))

In [ ]:
sorted(lr_coefs, key=lambda x: abs(x[1]), reverse=True)

In [ ]:
print(metrics.classification_report(y_val, lr_best.predict(prepro(X_train, X_val, scale=True)[1]), sample_weight=weights[y_val.index]))

In [ ]:
print(metrics.fbeta_score(y_val, lr_best.predict(prepro(X_train, X_val, scale=True)[1]), beta=0.5, sample_weight=weights[y_val.index]))

## Tune Hyperparameters for Random Forest

In [ ]:
n_estimators = [100]
criterion = ["gini"]#, "entropy"]
#max_depth = [None, 3, 5]
#min_samples_split = [2, 3]
#min_samples_leaf = [1, 2, 3]

max_depth = [None, 3]
min_samples_split = [2]
min_samples_leaf [1]
max_features = ["auto", "log2"]
n_jobs = [-1]

class_weight = [None, "balanced"]#, "balanced_subsample"]

keywords = ["n_estimators", "criterion", "max_depth", "min_samples_split", "min_samples_leaf", "max_features", "class_weight", "n_jobs"]
param_grid = product(n_estimators, criterion, max_depth, min_samples_split, min_samples_leaf, max_features, class_weight, n_jobs)

#samplers = [None] + [SMOTE(random_state=0, k_neighbors=k) for k in [2, 3, 5, 7, 10]] + [ADASYN(random_state=0, n_neighbors=n) for n in [2, 3, 5, 7, 10]]
samplers = [None] + [SMOTE(random_state=0, k_neighbors=k) for k in [3, 5]]
scale = [False]

prod_size = len(n_estimators)*len(criterion)*len(max_depth)*len(min_samples_split)*len(min_samples_leaf)*len(max_features)*len(class_weight)*len(samplers)*len(scale)
print_prog = (5, prod_size)

sc_rf = tune_hyper(X_train, y_train, RandomForestClassifier, keywords, param_grid, weights, samplers=samplers, scaling=scale, print_prog=print_prog)

In [ ]:
get_best_params(sc_rf, "fp5")

In [ ]:
keywords_rf = ["n_estimators", "criterion", "max_depth", "min_samples_split", "min_samples_leaf", "max_features", "class_weight", "n_jobs"]
rf_best = train_best_model(X_train, y_train, RandomForestClassifier, keywords_rf, sc_rf, "fp5")

In [ ]:
cols = prepro(X_train)[0].columns
rf_coefs = list(zip(cols, rf_best.feature_importances_))
rf_coefs

In [ ]:
print(metrics.fbeta_score(y_val, rf_best.predict(prepro(X_train, X_val, scale=False)[1]), beta=0.5, sample_weight=weights[y_val.index]))
print(metrics.classification_report(y_val, rf_best.predict(prepro(X_train, X_val, scale=False)[1]), sample_weight=weights[y_val.index]))

## Tune Hyperparameters for KNN

In [ ]:
n_neighbors = [5, 10, 20, 25, 35, 50, 75, 100]
weights_knn = ["uniform", "distance"]
algorithm = ["auto"]
p = [1, 2, 3, 4]

keywords = ["n_neighbors", "weights", "algorithm", "p"]
param_grid = product(n_neighbors, weights_knn, algorithm, p)

samplers = [None] + [SMOTE(random_state=0, k_neighbors=k) for k in [3, 5, 7]]
scale = [True]

prod_size = len(n_neighbors)*len(weights_knn)*len(algorithm)*len(p)*len(samplers)*len(scale)
print_prog = (10, prod_size)

sc_knn = tune_hyper(X_train, y_train, KNeighborsClassifier, keywords, param_grid, weights, randomized=False, samplers=samplers, scaling=scale, print_prog=print_prog)

In [ ]:
get_best_params(sc_knn, "fp5")

## Tune Hyperparameters for SVC

In [ ]:
C = [0.1, 0.5, 1, 5]
kernel = ["linear", "poly", "rbf", "sigmoid"]
gamma = ["scale", "auto"]
class_weight = [None, "balanced"]
probability = [False]

keywords = ["C", "kernel", "gamma", "class_weight", "probability"]
param_grid = product(C, kernel, gamma, class_weight, probability)

samplers = [None] + [SMOTE(random_state=0, k_neighbors=k) for k in [5, 7]]
scale = [True]

prod_size = len(C)*len(kernel)*len(gamma)*len(class_weight)*len(samplers)*len(scale)
print_prog = (1, prod_size)

sc_svc = tune_hyper(X_train, y_train, SVC, keywords, param_grid, samplers=samplers, scaling=scale, print_prog=print_prog)

In [ ]:
get_best_params(sc_svc, "fp5")

## Tune Hyperparameters for Adaboost

In [ ]:
n_estimators = [25, 50, 100, 150]
learning_rate = [1, 0.8, 0.5]
keywords = ["n_estimators", "learning_rate"]
param_grid = product(n_estimators, learning_rate)

samplers = [None] + [SMOTE(random_state=0, k_neighbors=k) for k in [2, 3, 5, 7, 10]] + [ADASYN(random_state=0, n_neighbors=n) for n in [2, 3, 5, 7, 10]]
scale = [False]

prod_size = len(n_estimators)*len(learning_rate)*len(samplers)*len(scale)
print_prog = (10, prod_size)

sc_ada = tune_hyper(X_train, y_train, AdaBoostClassifier, keywords, param_grid, samplers=samplers, scaling=scale, print_prog=print_prog)

In [ ]:
get_best_params(sc_ada, "f1")

In [ ]:
sort_scores_by_metric(sc_ada, "f1") 

## Tune Hyperparameters for XGBoost

In [ ]:
n_estimators = [30000]
max_depth = [2, 3, 5, 7]
objective = ["reg:squarederror", "binary:logistic", "reg:gamma", "reg:tweedie"]

learning_rate =[0.05, 0.1, 0.15]
subsample = [0.25, 0.5, 0.75, 1]
colsample_bytree = [0.2, 0.4, 0.7, 1.0]
lam = [0.1, 0.5, 1, 10, 100, 1000]
    
eval_metric = ["logloss"] 


max_depth = [2, 3]
learning_rate = [0.1]
lam = [0.5, 1, 5, 10, 100]
subsample = [0.5, 0.8, 1]
colsample_bytree = [0.5, 0.8, 1]

keywords = ["n_estimators", "max_depth", "objective", "learning_rate", "subsample", "colsample_bytree", "lambda", "eval_metric"]
param_grid = product(n_estimators, max_depth, objective, learning_rate, subsample, colsample_bytree, lam, eval_metric)


samplers = [None] + [SMOTE(random_state=0, k_neighbors=k) for k in [2, 3, 5, 7, 10]] + [ADASYN(random_state=0, n_neighbors=n) for n in [2, 3, 5, 7, 10]]
scale = [False]

samplers = [None] + [SMOTE(random_state=0, k_neighbors=k) for k in [5, 7]] + [ADASYN(random_state=0, n_neighbors=n) for n in [5, 7]]
scale = [False]

prod_size = len(n_estimators)*len(max_depth)*len(objective)*len(learning_rate)*len(subsample)*len(colsample_bytree)*len(lam)*len(samplers)
print_prog = (100, prod_size)

sc_xgb = tune_hyper(X_train, y_train, XGBClassifier, keywords, param_grid, samplers=samplers, scaling=scale, print_prog=print_prog)

In [ ]:
#keywords = ["n_estimators", "max_depth", "objective", "learning_rate", "subsample", "colsample_bytree", "lambda", "eval_metric"]
#sort_scores_by_metric(sc_xgb, "f1")

## Train Models with Tuned Hyperparameters

In [ ]:
keywords_xgb = ["n_estimators", "max_depth", "objective", "learning_rate", "subsample", "colsample_bytree", "lambda", "eval_metric"]
#xgb_best = train_best_model(X_train, y_train, XGBClassifier, keywords_xgb, sc_xgb, "f1")
# to train xgb i can either train-test-split or use their built-in cv function
b_p = get_best_params(sc_xgb, "f1")
clf_params, sampler, scale = b_p[0]

X_tr, _ = prepro(X_train, scale=scale)
if sampler:
    X_tr, y_tr = sampler.fit_sample(X_tr, y_train)
    
kwargs = dict(zip(keywords_xgb[1:], clf_params[1:]))
kwargs["random_state"] = 0
skf = StratifiedKFold(n_splits = 5, shuffle=True, random_state=0)
dtrain = xgb.DMatrix(X_tr, label=y_tr.to_numpy())

cv_results = xgb.cv(
    kwargs,
    dtrain,
    num_boost_round=30000,
    seed=0,
    folds=skf,
    metrics={'logloss'},
    early_stopping_rounds=10,
    verbose_eval=20
)
# is this not training properly for a classifier vs regressor? why is predict yielding numbers?
# how do i make it into a scikitlearn compatible thing




In [ ]:
xgb_best = xgb.train(kwargs, dtrain, num_boost_round=len(cv_results))

In [ ]:
keywords_lr = ["C", "solver", "penalty", "class_weight"]
lr_best = train_best_model(X_train, y_train, LogisticRegression, keywords_lr, sc_lr, "fp5")
keywords_rf = ["n_estimators", "criterion", "max_depth", "min_samples_split", "min_samples_leaf", "max_features", "class_weight", "n_jobs"]
rf_best = train_best_model(X_train, y_train, RandomForestClassifier, keywords_rf, sc_rf, "fp5")

In [ ]:

#keywords_knn = ["n_neighbors", "weights", "algorithm", "p"]
#knn_best = train_best_model(X_train, y_train, KNeighborsClassifier, keywords_knn, sc_knn, "fp5", randomized=False)
keywords_svc = ["C", "kernel", "gamma", "class_weight", "probability"]
svc_best = train_best_model(X_train, y_train, SVC, keywords_svc, sc_svc, "fp5")


In [ ]:
keywords_lr = ["C", "solver", "penalty", "class_weight"]
lr_best = train_best_model(X_train, y_train, LogisticRegression, keywords_lr, sc_lr, "fp5")
keywords_rf = ["n_estimators", "criterion", "max_depth", "min_samples_split", "min_samples_leaf", "max_features", "class_weight", "n_jobs"]
rf_best = train_best_model(X_train, y_train, RandomForestClassifier, keywords_rf, sc_rf, "fp5")

keywords_knn = ["n_neighbors", "weights", "algorithm", "p"]
knn_best = train_best_model(X_train, y_train, KNeighborsClassifier, keywords_knn, sc_knn, "fp5")

keywords_svc = ["C", "kernel", "gamma", "class_weight", "probability"]
svc_best = train_best_model(X_train, y_train, SVC, keywords_svc, sc_svc, "fp5")

keywords_ada = ["n_estimators", "learning_rate"]
ada_best = train_best_model(X_train, y_train, AdaBoostClassifier, keywords_ada, sc_ada, "fp5")

keywords_xgb = ["n_estimators", "max_depth", "objective", "learning_rate", "subsample", "colsample_bytree", "lambda", "eval_metric"]
xgb_best = train_best_model(X_train, y_train, XGBClassifier, keywords_xgb, sc_xgb, "fp5")

## Save Best Models and Score Dicts)

In [ ]:
#with open(f"models/lr_best.pickle", "wb") as pfile:
#    pickle.dump(lr_best, pfile)
#with open (f"models/lr_scores.pickle", "wb") as pfile:
#    pickle.dump(sc_lr, pfile)
#with open(f"models/rf_best.pickle", "wb") as pfile:
#    pickle.dump(rf_best, pfile)
#with open (f"models/rf_scores.pickle", "wb") as pfile:
#    pickle.dump(sc_rf, pfile)
#with open(f"models/knn_best.pickle", "wb") as pfile:
#    pickle.dump(knn_best, pfile)
#with open (f"models/knn_scores.pickle", "wb") as pfile:
#    pickle.dump(sc_knn, pfile)
#with open(f"models/svc_best.pickle", "wb") as pfile:
#    pickle.dump(svc_best, pfile)
#with open (f"models/svc_scores.pickle", "wb") as pfile:
#    pickle.dump(sc_svc, pfile)
#with open(f"models/ada_best.pickle", "wb") as pfile:
#    pickle.dump(ada_best, pfile)
#with open (f"models/ada_scores.pickle", "wb") as pfile:
#    pickle.dump(sc_ada, pfile)
#with open(f"models/xgb_best.pickle", "wb") as pfile:
#    pickle.dump(xgb_best, pfile)
#with open (f"models/xgb_scores.pickle", "wb") as pfile:
#    pickle.dump(sc_xgb, pfile)

In [ ]:
with open(f"models/lr_best.pickle", "rb") as pfile:
    lr_best = pickle.load(pfile)
lr_best.coef_

In [ ]:
with open(f"models/rf_best.pickle", "rb") as pfile:
    rf_best = pickle.load(pfile)
with open(f"models/rf_scores.pickle", "rb") as pfile:
    sc_rf = pickle.load(pfile)
with open(f"models/lr_best.pickle", "rb") as pfile:
    lr_best = pickle.load(pfile)
with open(f"models/lr_scores.pickle", "rb") as pfile:
    sc_lr = pickle.load(pfile)    
with open(f"models/knn_best.pickle", "rb") as pfile:
    knn_best = pickle.load(pfile)
with open(f"models/svc_best.pickle", "rb") as pfile:
    svc_best = pickle.load(pfile)   
with open(f"models/ada_best.pickle", "rb") as pfile:
    ada_best = pickle.load(pfile)


In [ ]:
y_pred = ada_best.predict(prepro(X_train, X_val, scale=True)[1])
print(metrics.fbeta_score(y_val, y_pred, beta=0.5))
print(metrics.precision_score(y_val, y_pred))
print(metrics.recall_score(y_val, y_pred))

In [ ]:
get_best_params(sc_lr, "fp5")

In [ ]:
get_best_params(sc_lr, "fp5")

In [ ]:
fig, ax = plt.subplots(figsize=(20, 20))
sns.set(font_scale=2.8)
sns.set_style("white")
metrics.plot_confusion_matrix(rf_best,prepro(X_train, X_val, scale=False)[1], y_val, sample_weight=weights[y_val.index], normalize="all", cmap=plt.cm.Blues,ax=ax)
ax.set_xlabel('Predicted Status', fontsize=44, labelpad=20);
ax.set_ylabel('True Status', fontsize=44,labelpad=20);
ax.set_title('Confusion Matrix, Random Forest Model', pad=40, fontsize=50);
ax.xaxis.set_ticklabels(["No Bachelor's", "Bachelor's"], fontsize=40);
ax.yaxis.set_ticklabels(["No Bachelor's", "Bachelor's"], rotation=90,fontsize=40, va="center");
plt.show()

In [ ]:

y_pred_lr = lr_best.predict(prepro(X_train, X_val, scale=True)[1])
print(metrics.classification_report(y_val, y_pred_lr))
get_best_params(sc_lr, "fp5")

In [ ]:

y_pred_rf = rf_best.predict(prepro(X_train, X_val, scale=False)[1])
print(metrics.classification_report(y_val, y_pred_rf, sample_weight=weights[y_val.index]))
get_best_params(sc_rf, "fp5")

In [ ]:
metrics.fbeta_score(y_val, y_pred_rf, beta=0.5, sample_weight=weights[y_val.index])

In [ ]:
(1+0.5*0.5)*(0.7444897*0.68576744)/(0.5*0.5*0.7444897+0.68576744)

In [ ]:
metrics.fbeta_score(y_val, y_pred_rf, beta=0.5, sample_weight=weights[y_val.index])
metrics.precision_score(y_val, y_pred_rf, sample_weight=weights[y_val.index])
metrics.recall_score(y_val, y_pred_rf, sample_weight=weights[y_val.index])

In [ ]:
y.value_counts()

In [ ]:
sc_rf[((100, 'gini', None, 2, 1, 'auto', None, -1), None, False)]

In [ ]:
array = [[172, 22],[39,58]]
df_cm = pd.DataFrame(array, index = [i for i in ["Bachelor's", "No Bachelor's"]],
              columns = [i for i in ["Bachelor's", "No Bachelor's"]])
plt.figure(figsize = (10,7))
sns.heatmap(df_cm, annot=True,cmap="OrRd")

In [ ]:

cols = prepro(X_train)[0].columns
lr_coefs = list(zip(cols, a.coef_[0]))
pd.DataFrame(lr_coefs).to_csv('site/feature_importances.csv', header=True, index=False)

pd.read_csv("site/feature_importances.csv")

In [ ]:
with open(f"site/feature_importances.pickle", "wb") as pfile:
    pickle.dump(lr_coefs, pfile)
with open(f"site/feature_importances.pickle", "rb") as pfile:
    print(pickle.load(pfile))

## Compare Best Models and Plot Metrics

In [ ]:
# might be best to make these pipelines that automatically scale or don't scale

In [ ]:
model_names = ["lr_best", "rf_best", "knn_best", "svc_best", "ada_best", "xgb_best"]
to_scale = [True, False, True, True, False, False]
models = []

for model in model_names:
    with open(f"models/{model}.pickle", "rb") as pfile:
        exec(f"{model} = pickle.load(pfile)")
        exec(f"models.append({model})")
# make sure to check whether log is scaled - perhaps automate it from best scores

In [ ]:
lr_best.coef_[0]

In [ ]:
cols = prepro(X_train)[0].columns
lr_coefs = list(zip(cols, lr_best.coef_[0]))
lr_coefs

In [ ]:
svc_coefs = list(zip(cols, svc_best.coef_[0]))
svc_coefs

In [ ]:
sns.countplot(data=data, x="region", hue="grad_bach")

In [ ]:
X_tr_prepro_sc, X_v_prepro_sc = prepro(X_train, X_val, scale=True)
X_tr_prepro, X_v_prepro = prepro(X_train, X_val)

for model, model_name, scale in zip(models, model_names, to_scale):
    X_v_select = X_v_prepro_sc if scale else X_v_prepro
    if model_name == "xgb_best":
        X_v_select = xgb.DMatrix(X_v_select)
    y_pred_lr = model.predict(X_v_select)
    print(model_name, "\n", metrics.classification_report(y_val, np.round(y_pred_lr)))

In [ ]:
y_pred_lr = xgb_best.predict(xgb.DMatrix(X_v_prepro))
print(metrics.classification_report(y_val, np.round(y_pred_lr)))

In [ ]:
y_pred_lr = xgb_best.predict(xgb.DMatrix(X_v_prepro))
y_probs_lr = xgb_best.predict_proba(xgb.DMatrix(X_v_prepro))

print(metrics.classification_report(y_train, xgb_best.predict(X_tr_prepro)))
print(metrics.classification_report(y_val, y_pred_lr))

In [ ]:
y_pred_lr = lr_best.predict(X_v_prepro_sc)
y_probs_lr = lr_best.predict_proba(X_v_prepro_sc)
print(metrics.classification_report(y_val, y_pred_lr))

In [ ]:
sklearn.metrics.plot_precision_recall_curve(lr_best, X_v_prepro_sc, y_val)
sklearn.metrics.plot_roc_curve(lr_best, X_v_prepro_sc, y_val)

In [ ]:
p, r, th = metrics.precision_recall_curve(y_true, y_probs[:, 1])

In [ ]:
y_pred_rf = rf_best.predict(X_v_prepro)
y_probs_rf = rf_best.predict_proba(X_v_prepro)
print(metrics.classification_report(y_val, y_pred_rf))

In [ ]:
sklearn.metrics.plot_precision_recall_curve(rf_best, X_v_prepro, y_val)
sklearn.metrics.plot_roc_curve(rf_best, X_v_prepro, y_val)

In [ ]:
y_pred_rf = knn_best.predict(X_v_prepro_sc)
y_probs_rf = knn_best.predict_proba(X_v_prepro_sc)
print(metrics.classification_report(y_val, y_pred_rf))

In [ ]:
# todo:
# plot precision-recall curves for each best model
# analyze feature importances
# fix xgboost, retrain it as an sklearn-compatible model


# workflow:
# fix the preprocessing logic
# initialize k-folds and samplers outside instead of making tons of them (may speed things up?)

# extra:
# perhaps make decision boundary plot
# for final xgb training do i pass it an eval set?



In [ ]:
# make an easier way to just do X_val_prepro